# Finding Missing Person

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
#import keras
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tensorflow.__version__

'2.6.1'

In [3]:
tensorflow.keras.__version__

'2.6.0'

### Image Data Agumentation

In [4]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('../dataset/trainset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='binary')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('../dataset/testset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='binary') 

Found 174 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [6]:
print(x_train.class_indices)#checking the number of classes

{'Found Missing': 0, 'Normal': 1}


In [7]:
print(x_test.class_indices)#checking the number of classes

{'Found Missing': 0, 'Normal': 1}


In [8]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 80, 1: 94})

In [9]:
from collections import Counter as c
c(x_test .labels)

Counter({0: 20, 1: 40})

### Creating the model

In [10]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid')) # softmax for more than 2



In [11]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

### Compiling the model

In [12]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [13]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

C:\Users\varan\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
35/35 [==============================] - 5s 118ms/step - loss: 0.7067 - accuracy: 0.5172 - val_loss: 0.6751 - val_accuracy: 0.6667
Epoch 2/20
35/35 [==============================] - 4s 111ms/step - loss: 0.6854 - accuracy: 0.5460 - val_loss: 0.7271 - val_accuracy: 0.6833
Epoch 3/20
35/35 [==============================] - 4s 117ms/step - loss: 0.6761 - accuracy: 0.5517 - val_loss: 0.7382 - val_accuracy: 0.4833
Epoch 4/20
35/35 [==============================] - 4s 100ms/step - loss: 0.6709 - accuracy: 0.6092 - val_loss: 0.7220 - val_accuracy: 0.6000
Epoch 5/20
35/35 [==============================] - 4s 112ms/step - loss: 0.6674 - accuracy: 0.6092 - val_loss: 0.9408 - val_accuracy: 0.2833
Epoch 6/20
35/35 [==============================] - 4s 101ms/step - loss: 0.6528 - accuracy: 0.6322 - val_loss: 0.8242 - val_accuracy: 0.3667
Epoch 7/20
35/35 [==============================] - 4s 101ms/step - loss: 0.6132 - accuracy: 0.6724 - val_loss: 0.7850 - val_accuracy: 0.5167
Epoch 

### Saving our model

In [14]:
# Save the model
classifier.save('missing_person.h5')

In [15]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [16]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("missing_person.h5") #loading the model for testing

In [25]:
img = image.load_img(r"C:\Users\varan\OneDrive\Desktop\dmp\missing_person_updated\dataset\trainset\Normal\Akash-ambani-wedding_14_20190311.jpg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
#pred=model.predict_classes(x)
y=int (model.predict(x))
print(y)
#pred = np.argmax(y,axis=1)#predicting the classes
#pred

1


In [26]:
index=['Found Missing', 'Normal']
result=str(index[y])
result

'Normal'